# IMPORTAÇÃO DE BIBLIOTECAS E DRIVE

As 4 primeiras linhas foram adicionadas antes de saber o que seria necessário durante a execução do TCC. As seguintes foram acrescentadas assim que necessário.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.linear_model import ElasticNetCV

Montagem do Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# IMPORTAÇÃO DE DADOS

Importação do Dataframe final do Notebook 1 que reune em um só CSV os 19 Dataframes iniciais


In [3]:
df_base = pd.read_csv("/content/drive/MyDrive/tcc/dataframes/df_base.csv")

# PRÉ-ANÁLISE DE DADOS

O Dataframe nesse momento apresenta 130 colunas. Além de fazer o tratamento e processamento de dados, a opção foi fazer simultaneamente uma pré-análise para evitar a necessidade de fazer tratamento de tamanha quantidade de dados.

In [4]:
df_base

,codigo,aids,animais,dengue,hanseniase,hepatites,sadquirida,scongenita,sgestante,toxico,...,"Impostos, líquidos de subsídios, sobre produtos, a preços correntes (R$ 1.000)","Produto Interno Bruto, a preços correntes (R$ 1.000)","Produto Interno Bruto per capita, a preços correntes (R$ 1,00)",Atividade com maior valor adicionado bruto,Atividade com segundo maior valor adicionado bruto,Atividade com terceiro maior valor adicionado bruto,populacao,latitude,longitude,regiao
0,110001,1,192,833,51,41,28,2,12,186,...,"29,351","495,629","21,600.73","Administração, defesa, educação e saúde públic...",Demais serviços,"Pecuária, inclusive apoio à pecuária",22945,-11.92870,-61.9958,N
1,110037,1,199,90,86,12,13,1,2,31,...,"11,248","288,681","21,802.07","Administração, defesa, educação e saúde públic...","Agricultura, inclusive apoio à agricultura e a...","Pecuária, inclusive apoio à pecuária",13241,-12.14050,-61.8379,N
2,110040,5,62,65,43,66,18,2,11,18,...,"18,150","358,464","16,728.75","Administração, defesa, educação e saúde públic...","Pecuária, inclusive apoio à pecuária",Demais serviços,21428,-9.71470,-63.3193,N
3,110034,5,78,126,24,8,16,1,7,35,...,"10,423","260,348","18,065.91","Administração, defesa, educação e saúde públic...","Pecuária, inclusive apoio à pecuária",Demais serviços,14411,-11.34670,-62.2852,N
4,110002,121,252,198,224,240,296,8,89,321,...,"284,398","2,578,830","23,908.38",Demais serviços,"Administração, defesa, educação e saúde públic...",Comércio e reparação de veículos automotores e...,107863,-9.90612,-63.0330,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,522145,0,24,115,12,0,0,0,1,3,...,"2,444","51,875","14,821.32","Administração, defesa, educação e saúde públic...","Pecuária, inclusive apoio à pecuária",Demais serviços,3500,-13.50830,-48.7421,CO
5566,522157,0,17,137,11,1,1,1,2,0,...,"1,956","47,960","16,804.54","Administração, defesa, educação e saúde públic...","Pecuária, inclusive apoio à pecuária",Demais serviços,2854,-14.28400,-49.9205,CO
5567,220460,0,0,14,6,2,0,1,2,1,...,"1,292","30,136","7,773.10","Administração, defesa, educação e saúde públic...",Demais serviços,Comércio e reparação de veículos automotores e...,3877,-5.98900,-42.5601,NE
5568,220590,0,0,21,5,0,7,3,3,3,...,"2,888","51,080","9,549.44","Administração, defesa, educação e saúde públic...",Demais serviços,Comércio e reparação de veículos automotores e...,5349,-8.00434,-43.8723,NE


In [5]:
correlacao = df_base.corr()

In [6]:
correlacao_doencas = correlacao[["aids", "animais", "dengue", "hanseniase", "hepatites", "sadquirida", "scongenita", "sgestante", "toxico", "tuberculose", "autoprovocada_sim", "autoprovocada_não", "autoprovocada_ignorado", "autoprovocada_branco", "violencia" ]].transpose()


In [7]:
correlacao_doencas.style.background_gradient(axis=None)

,codigo,aids,animais,dengue,hanseniase,hepatites,sadquirida,scongenita,sgestante,toxico,tuberculose,autoprovocada_sim,autoprovocada_não,autoprovocada_ignorado,autoprovocada_branco,violencia,ACADEMIA DA SAÚDE,CENTRAL DE REGUALAÇÃO,CENTRAL DE REGULAÇÃO MÉDICA DAS URGÊNCIAS,CENTRO DE APOIO A SAÚDE DA FAMÍLIA-CASF,CENTRO DE ATENÇÃO HEMOTERÁPICA E/OU HEMATOLÓGICA,CENTRO DE ATENÇÃO PSICOSSOCIAL-CAPS,CENTRO DE PARTO NORMAL,CENTRO DE SAUDE/UNIDADE BASICA DE SAUDE,CENTRAL DE REGULACAO DE SERVICOS DE SAUDE,CENTRAL DE NOTIF. CAPTAÇÃO E DISTR. ÓRGÃOS ESTADUA,CLINICA ESPECIALIZADA/AMBULATORIO ESPECIALIZADO,CONSULTORIO,COOPERATIVA,FARMACIA,HOSPITAL ESPECIALIZADO,HOSPITAL GERAL,HOSPITAL DIA,LABORATORIO CENTRAL DE SAUDE PUBLICA - LACEN,LABORATORIO DE SAUDE PUBLICA,POLICLINICA,OFICINA ORTOPEDICA,POSTO DE SAUDE,PRONTO ANTEDIMENTO,PRONTO SOCORRO ESPECIALIZADO,PRONTO SOCORRO GERAL,SECRETARIA DE SAUDE,SERVICO DE ATENCAO DOMICILIAR ISOLADO(HOME CARE),UNIDADE DE ATENÇÃO À SAÚDE INDÍGENA,UNIDADE DE ATENCAO EM REGIME RESIDENCIAL,UNIDADE DE SERVICO DE APOIO DE DIAGNOSE E TERAPIA,UNIDADE DE VIGILANCIA EM SAUDE,UNIDADE MISTA,UNIDADE MOVEL DE NIVEL PRE-HOSP-URGENCIA/EMERGENCI,UNIDADE MOVEL FLUVIAL,UNIDADE MOVEL TERRESTRE,TELESAÚDE,POLO PREV.DE DOENCAS E AGRAVOS E PROMOCAO DA SAUDE,total_de_estabelecimentos,mortalidade_geral_2015,mortalidade_geral_2016,mortalidade_geral_2017,mortalidade_geral_2018,mortalidade_geral_2019,mortalidade_infantil_2015,mortalidade_infantil_2016,mortalidade_infantil_2017,mortalidade_infantil_2018,mortalidade_infantil_2019,nascimentos_2015,nascimentos_2016,nascimentos_2017,nascimentos_2018,nascimentos_2019,Assistente Social,Bioquímico/farmacêutico,Cirurgião Geral,Clínico Geral,Enfermeiro,Fisioterapeuta,Fonoaudiólogo,Gineco Obstetra,Médico de Família,Nutricionista,Odontólogo,Pediatra,Psicólogo,Psiquiatra,Radiologista,Sanitarista,Outras especialidades médicas,Outras ocupações de nível superior relac à Saúde,total_de_profissionais,codigo_ibge,Código da Mesorregião,Código da Microrregião,Código da Região Geográfica Imediata,Código da Região Geográfica Intermediária,Código Concentração Urbana,Código Arranjo Populacional,Código da Região Rural,populacao,latitude,longitude
aids,-0.000393,1.000000,0.353943,0.473516,0.494419,0.882588,0.890405,0.927438,0.947267,0.819689,0.965127,0.835300,0.892537,0.470651,0.474621,0.895069,0.217913,0.443236,0.419071,0.005147,0.787982,0.862034,0.395645,0.849067,0.165964,0.669054,0.904479,0.894924,0.428675,0.486165,0.835436,0.912469,0.734353,0.193280,0.485820,0.540392,0.287645,0.100219,0.791849,0.672322,0.625777,0.701251,0.833666,0.043587,0.036790,0.854003,0.481002,0.106302,0.812979,0.215147,0.677444,0.729266,0.292571,0.919828,0.960956,0.960476,0.961585,0.960743,0.960057,0.956155,0.951728,0.946636,0.948483,0.948534,0.954312,0.951865,0.950929,0.950473,0.948808,0.943329,0.946960,0.917641,0.913980,0.943321,0.923590,0.917916,0.921232,0.927888,0.942185,0.912140,0.932139,0.910225,0.878052,0.922354,0.836052,0.911708,0.890674,0.934611,-0.000393,-0.000114,-0.000238,-0.000304,-0.000483,-0.083733,-0.039926,-0.000930,0.955766,-0.013581,-0.001286
animais,-0.030917,0.353943,1.000000,0.414406,0.411708,0.250601,0.257301,0.406460,0.256813,0.422228,0.277976,0.432022,0.365492,0.174892,0.309730,0.384053,0.263407,0.301674,0.367121,0.025623,0.259156,0.395981,0.099807,0.520458,0.196267,0.424253,0.410838,0.359085,0.431658,0.154243,0.478081,0.422594,0.348190,0.137830,0.168016,0.370445,0.198206,0.045568,0.381620,0.417804,0.171844,0.424130,0.421244,0.042022,0.009992,0.411723,0.275921,0.053777,0.420126,0.018388,0.350294,0.400079,0.235539,0.399233,0.348568,0.345876,0.355124,0.343617,0.342057,0.385732,0.376192,0.382700,0.379658,0.380272,0.382540,0.380508,0.382878,0.389138,0.389419,0.444167,0.372867,0.396050,0.411770,0.361068,0.415256,0.425164,0.363958,0.385725,0.403611,0.395805,0.388601,0.415224,0.298443,0.339535,0.157018,0.318721,0.359692,0.375665,-0.030917,-0.031295,-0.031383,-0.031424,-0.031424,-0.152247,-0.099740,-0.030321,0.377966,0.0

Antes desta análise o autor preservou dados diferenciando as diferentes formas de violência (autoprovocada ou não, além de branco e ignorado). Isso foi feito porque naquele momento, imaginava que poderia somar os brancos e ignorados com os casos de violência não autoprovocada pois esses teriam alta correlação (não é verdade como percebe-se no notebook). Também, o autor considera que a fração de 7.3% (% de violência brancos ou ignorados) é muito significativa para simplesmente desprezar os dados. No entanto, a verdade é que os dados em branco e ignorados não possuem muito valor estatístico para qualificar o tipo de violência. 

In [8]:
correlacao_estabelecimentos = correlacao[["total_de_estabelecimentos"]].transpose()
correlacao_estabelecimentos.style.background_gradient(axis=None)


,codigo,aids,animais,dengue,hanseniase,hepatites,sadquirida,scongenita,sgestante,toxico,tuberculose,autoprovocada_sim,autoprovocada_não,autoprovocada_ignorado,autoprovocada_branco,violencia,ACADEMIA DA SAÚDE,CENTRAL DE REGUALAÇÃO,CENTRAL DE REGULAÇÃO MÉDICA DAS URGÊNCIAS,CENTRO DE APOIO A SAÚDE DA FAMÍLIA-CASF,CENTRO DE ATENÇÃO HEMOTERÁPICA E/OU HEMATOLÓGICA,CENTRO DE ATENÇÃO PSICOSSOCIAL-CAPS,CENTRO DE PARTO NORMAL,CENTRO DE SAUDE/UNIDADE BASICA DE SAUDE,CENTRAL DE REGULACAO DE SERVICOS DE SAUDE,CENTRAL DE NOTIF. CAPTAÇÃO E DISTR. ÓRGÃOS ESTADUA,CLINICA ESPECIALIZADA/AMBULATORIO ESPECIALIZADO,CONSULTORIO,COOPERATIVA,FARMACIA,HOSPITAL ESPECIALIZADO,HOSPITAL GERAL,HOSPITAL DIA,LABORATORIO CENTRAL DE SAUDE PUBLICA - LACEN,LABORATORIO DE SAUDE PUBLICA,POLICLINICA,OFICINA ORTOPEDICA,POSTO DE SAUDE,PRONTO ANTEDIMENTO,PRONTO SOCORRO ESPECIALIZADO,PRONTO SOCORRO GERAL,SECRETARIA DE SAUDE,SERVICO DE ATENCAO DOMICILIAR ISOLADO(HOME CARE),UNIDADE DE ATENÇÃO À SAÚDE INDÍGENA,UNIDADE DE ATENCAO EM REGIME RESIDENCIAL,UNIDADE DE SERVICO DE APOIO DE DIAGNOSE E TERAPIA,UNIDADE DE VIGILANCIA EM SAUDE,UNIDADE MISTA,UNIDADE MOVEL DE NIVEL PRE-HOSP-URGENCIA/EMERGENCI,UNIDADE MOVEL FLUVIAL,UNIDADE MOVEL TERRESTRE,TELESAÚDE,POLO PREV.DE DOENCAS E AGRAVOS E PROMOCAO DA SAUDE,total_de_estabelecimentos,mortalidade_geral_2015,mortalidade_geral_2016,mortalidade_geral_2017,mortalidade_geral_2018,mortalidade_geral_2019,mortalidade_infantil_2015,mortalidade_infantil_2016,mortalidade_infantil_2017,mortalidade_infantil_2018,mortalidade_infantil_2019,nascimentos_2015,nascimentos_2016,nascimentos_2017,nascimentos_2018,nascimentos_2019,Assistente Social,Bioquímico/farmacêutico,Cirurgião Geral,Clínico Geral,Enfermeiro,Fisioterapeuta,Fonoaudiólogo,Gineco Obstetra,Médico de Família,Nutricionista,Odontólogo,Pediatra,Psicólogo,Psiquiatra,Radiologista,Sanitarista,Outras especialidades médicas,Outras ocupações de nível superior relac à Saúde,total_de_profissionais,codigo_ibge,Código da Mesorregião,Código da Microrregião,Código da Região Geográfica Imediata,Código da Região Geográfica Intermediária,Código Concentração Urbana,Código Arranjo Populacional,Código da Região Rural,populacao,latitude,longitude
total_de_estabelecimentos,0.025291,0.919828,0.399233,0.548928,0.392661,0.893304,0.941164,0.824624,0.902663,0.900494,0.876032,0.916868,0.928598,0.461102,0.511112,0.937807,0.244856,0.457170,0.398424,0.002179,0.859699,0.913307,0.416723,0.835064,0.188984,0.670852,0.955074,0.990967,0.414330,0.557270,0.795117,0.935888,0.784536,0.201556,0.435693,0.602591,0.290739,0.092831,0.748480,0.630708,0.640002,0.668354,0.903553,0.037103,0.037572,0.913342,0.510978,0.085259,0.828162,0.037535,0.729811,0.716851,0.350282,1.000000,0.956662,0.955005,0.958005,0.955926,0.954010,0.920417,0.919634,0.917571,0.914574,0.914620,0.945953,0.945750,0.946032,0.944725,0.943707,0.937876,0.971201,0.942027,0.962794,0.966515,0.969713,0.970522,0.958459,0.923434,0.937771,0.985076,0.960002,0.981479,0.951977,0.957998,0.867839,0.966220,0.953669,0.977976,0.025291,0.025171,0.025024,0.024926,0.024856,-0.019014,-0.005755,0.025185,0.958985,-0.047458,-0.002583


A opção por preservar somente o número total de estabelecimentos foi porque ao avaliar algumas cidades individualmente, percebeu números absolutamente incompatíveis com sua dimensão. O autor cogitou retirar completamente este dado do estudo mas preservou o total por se tratar de apenas uma coluna, sem maiores impactos de processamento no momento de realizar o modelo de machine learning.

In [9]:
correlacao_profissionais = correlacao[["total_de_profissionais"]].transpose()
correlacao_profissionais.style.background_gradient(axis=None)

,codigo,aids,animais,dengue,hanseniase,hepatites,sadquirida,scongenita,sgestante,toxico,tuberculose,autoprovocada_sim,autoprovocada_não,autoprovocada_ignorado,autoprovocada_branco,violencia,ACADEMIA DA SAÚDE,CENTRAL DE REGUALAÇÃO,CENTRAL DE REGULAÇÃO MÉDICA DAS URGÊNCIAS,CENTRO DE APOIO A SAÚDE DA FAMÍLIA-CASF,CENTRO DE ATENÇÃO HEMOTERÁPICA E/OU HEMATOLÓGICA,CENTRO DE ATENÇÃO PSICOSSOCIAL-CAPS,CENTRO DE PARTO NORMAL,CENTRO DE SAUDE/UNIDADE BASICA DE SAUDE,CENTRAL DE REGULACAO DE SERVICOS DE SAUDE,CENTRAL DE NOTIF. CAPTAÇÃO E DISTR. ÓRGÃOS ESTADUA,CLINICA ESPECIALIZADA/AMBULATORIO ESPECIALIZADO,CONSULTORIO,COOPERATIVA,FARMACIA,HOSPITAL ESPECIALIZADO,HOSPITAL GERAL,HOSPITAL DIA,LABORATORIO CENTRAL DE SAUDE PUBLICA - LACEN,LABORATORIO DE SAUDE PUBLICA,POLICLINICA,OFICINA ORTOPEDICA,POSTO DE SAUDE,PRONTO ANTEDIMENTO,PRONTO SOCORRO ESPECIALIZADO,PRONTO SOCORRO GERAL,SECRETARIA DE SAUDE,SERVICO DE ATENCAO DOMICILIAR ISOLADO(HOME CARE),UNIDADE DE ATENÇÃO À SAÚDE INDÍGENA,UNIDADE DE ATENCAO EM REGIME RESIDENCIAL,UNIDADE DE SERVICO DE APOIO DE DIAGNOSE E TERAPIA,UNIDADE DE VIGILANCIA EM SAUDE,UNIDADE MISTA,UNIDADE MOVEL DE NIVEL PRE-HOSP-URGENCIA/EMERGENCI,UNIDADE MOVEL FLUVIAL,UNIDADE MOVEL TERRESTRE,TELESAÚDE,POLO PREV.DE DOENCAS E AGRAVOS E PROMOCAO DA SAUDE,total_de_estabelecimentos,mortalidade_geral_2015,mortalidade_geral_2016,mortalidade_geral_2017,mortalidade_geral_2018,mortalidade_geral_2019,mortalidade_infantil_2015,mortalidade_infantil_2016,mortalidade_infantil_2017,mortalidade_infantil_2018,mortalidade_infantil_2019,nascimentos_2015,nascimentos_2016,nascimentos_2017,nascimentos_2018,nascimentos_2019,Assistente Social,Bioquímico/farmacêutico,Cirurgião Geral,Clínico Geral,Enfermeiro,Fisioterapeuta,Fonoaudiólogo,Gineco Obstetra,Médico de Família,Nutricionista,Odontólogo,Pediatra,Psicólogo,Psiquiatra,Radiologista,Sanitarista,Outras especialidades médicas,Outras ocupações de nível superior relac à Saúde,total_de_profissionais,codigo_ibge,Código da Mesorregião,Código da Microrregião,Código da Região Geográfica Imediata,Código da Região Geográfica Intermediária,Código Concentração Urbana,Código Arranjo Populacional,Código da Região Rural,populacao,latitude,longitude
total_de_profissionais,0.013879,0.934611,0.375665,0.562691,0.401227,0.899005,0.959247,0.837434,0.917831,0.923334,0.892503,0.914930,0.923532,0.476918,0.519765,0.935779,0.275922,0.443472,0.365512,-0.003553,0.889776,0.916713,0.453306,0.835213,0.190420,0.674860,0.968020,0.953656,0.408670,0.546380,0.797751,0.933303,0.825176,0.203129,0.430068,0.601672,0.288442,0.076671,0.751723,0.649846,0.657334,0.667772,0.885486,0.033140,0.023322,0.900749,0.525250,0.088968,0.863854,0.057980,0.731827,0.730688,0.321012,0.977976,0.969186,0.966901,0.969988,0.967802,0.966840,0.947479,0.947385,0.945362,0.944097,0.942456,0.970245,0.970071,0.970844,0.969527,0.968479,0.963484,0.987888,0.978382,0.976352,0.995049,0.991012,0.986824,0.991758,0.938404,0.967185,0.986094,0.986572,0.990406,0.974154,0.989744,0.893620,0.993053,0.975953,1.000000,0.013879,0.013872,0.013732,0.013643,0.013532,-0.030068,-0.015350,0.013569,0.980373,-0.029020,0.005731


A opção por manter somente o total de profissionais foi devido à alta correlação entre esse valor e o de cada profissional individualmente, de modo que optei por poupar processamento de dados.

In [10]:
for column in df_base.columns:
  print(column)

codigo
aids
animais
dengue
hanseniase
hepatites
sadquirida
scongenita
sgestante
toxico
tuberculose
autoprovocada_sim
autoprovocada_não
autoprovocada_ignorado
autoprovocada_branco
violencia
ACADEMIA DA SAÚDE
CENTRAL DE REGUALAÇÃO
CENTRAL DE REGULAÇÃO MÉDICA DAS URGÊNCIAS
CENTRO DE APOIO A SAÚDE DA FAMÍLIA-CASF
CENTRO DE ATENÇÃO HEMOTERÁPICA E/OU HEMATOLÓGICA
CENTRO DE ATENÇÃO PSICOSSOCIAL-CAPS
CENTRO DE PARTO NORMAL
CENTRO DE SAUDE/UNIDADE BASICA DE SAUDE
CENTRAL DE REGULACAO DE SERVICOS DE SAUDE
CENTRAL DE NOTIF. CAPTAÇÃO E DISTR. ÓRGÃOS ESTADUA
CLINICA ESPECIALIZADA/AMBULATORIO ESPECIALIZADO
CONSULTORIO
COOPERATIVA
FARMACIA
HOSPITAL ESPECIALIZADO
HOSPITAL GERAL
HOSPITAL DIA
LABORATORIO CENTRAL DE SAUDE PUBLICA - LACEN
LABORATORIO DE SAUDE PUBLICA
POLICLINICA
OFICINA ORTOPEDICA
POSTO DE SAUDE
PRONTO ANTEDIMENTO
PRONTO SOCORRO ESPECIALIZADO
PRONTO SOCORRO GERAL
SECRETARIA DE SAUDE
SERVICO DE ATENCAO DOMICILIAR ISOLADO(HOME CARE)
UNIDADE DE ATENÇÃO À SAÚDE INDÍGENA
UNIDADE DE ATENCAO EM 

In [11]:
df_base

,codigo,aids,animais,dengue,hanseniase,hepatites,sadquirida,scongenita,sgestante,toxico,...,"Impostos, líquidos de subsídios, sobre produtos, a preços correntes (R$ 1.000)","Produto Interno Bruto, a preços correntes (R$ 1.000)","Produto Interno Bruto per capita, a preços correntes (R$ 1,00)",Atividade com maior valor adicionado bruto,Atividade com segundo maior valor adicionado bruto,Atividade com terceiro maior valor adicionado bruto,populacao,latitude,longitude,regiao
0,110001,1,192,833,51,41,28,2,12,186,...,"29,351","495,629","21,600.73","Administração, defesa, educação e saúde públic...",Demais serviços,"Pecuária, inclusive apoio à pecuária",22945,-11.92870,-61.9958,N
1,110037,1,199,90,86,12,13,1,2,31,...,"11,248","288,681","21,802.07","Administração, defesa, educação e saúde públic...","Agricultura, inclusive apoio à agricultura e a...","Pecuária, inclusive apoio à pecuária",13241,-12.14050,-61.8379,N
2,110040,5,62,65,43,66,18,2,11,18,...,"18,150","358,464","16,728.75","Administração, defesa, educação e saúde públic...","Pecuária, inclusive apoio à pecuária",Demais serviços,21428,-9.71470,-63.3193,N
3,110034,5,78,126,24,8,16,1,7,35,...,"10,423","260,348","18,065.91","Administração, defesa, educação e saúde públic...","Pecuária, inclusive apoio à pecuária",Demais serviços,14411,-11.34670,-62.2852,N
4,110002,121,252,198,224,240,296,8,89,321,...,"284,398","2,578,830","23,908.38",Demais serviços,"Administração, defesa, educação e saúde públic...",Comércio e reparação de veículos automotores e...,107863,-9.90612,-63.0330,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,522145,0,24,115,12,0,0,0,1,3,...,"2,444","51,875","14,821.32","Administração, defesa, educação e saúde públic...","Pecuária, inclusive apoio à pecuária",Demais serviços,3500,-13.50830,-48.7421,CO
5566,522157,0,17,137,11,1,1,1,2,0,...,"1,956","47,960","16,804.54","Administração, defesa, educação e saúde públic...","Pecuária, inclusive apoio à pecuária",Demais serviços,2854,-14.28400,-49.9205,CO
5567,220460,0,0,14,6,2,0,1,2,1,...,"1,292","30,136","7,773.10","Administração, defesa, educação e saúde públic...",Demais serviços,Comércio e reparação de veículos automotores e...,3877,-5.98900,-42.5601,NE
5568,220590,0,0,21,5,0,7,3,3,3,...,"2,888","51,080","9,549.44","Administração, defesa, educação e saúde públic...",Demais serviços,Comércio e reparação de veículos automotores e...,5349,-8.00434,-43.8723,NE


A seguir é realizada uma grande exclusão de dados do Dataframe principal. As questões relacionadas à violência, estabelecimentos e profissionais já foram detalhadas acima. Importante explicar que a decisão de preservar os registros de região Metropolitana, Semiárido e Amazônia Legal foi tomada porque esses dados poderiam influenciar além da questão geográfica numérica (latitude e longitude) no momento da construção dos modelos de Machine Learning. Também optado por manter os valores de PIB por tipo de produção em cada município pois esse tipo de característica de cada município vai além da simples localização geográfica.

In [12]:
df_pre = df_base[["codigo", "aids", "animais", "dengue", "hanseniase", "hepatites", "sadquirida", "scongenita", "sgestante", 
 "toxico", "tuberculose", "autoprovocada_sim", "autoprovocada_não", "violencia", "total_de_estabelecimentos", 
 "mortalidade_geral_2015", "mortalidade_geral_2016", "mortalidade_geral_2017", "mortalidade_geral_2018", 
 "mortalidade_geral_2019", "mortalidade_infantil_2015", "mortalidade_infantil_2016", "mortalidade_infantil_2017",
 "mortalidade_infantil_2018", "mortalidade_infantil_2019", "nascimentos_2015", "nascimentos_2016", "nascimentos_2017",
 "nascimentos_2018", "nascimentos_2019", "total_de_profissionais", "estado", "Região Metropolitana",
 "Amazônia Legal", "Semiárido", "Valor adicionado bruto da Agropecuária, preços correntes(R$ 1.000)", 
 "Valor adicionado bruto da Indústria, a preços correntes (R$ 1.000)", 
 "Valor adicionado bruto dos Serviços, a preços correntes  - exceto Administração, defesa, educação e saúde públicas e seguridade social (R$ 1.000)",
 "Valor adicionado bruto da Administração, defesa, educação e saúde públicas e seguridade social, a preços correntes (R$ 1.000)", 
 "Impostos, líquidos de subsídios, sobre produtos, a preços correntes (R$ 1.000)",
 "Produto Interno Bruto, a preços correntes (R$ 1.000)", "Produto Interno Bruto per capita, a preços correntes (R$ 1,00)",
 "populacao", "latitude", "longitude", "regiao"]]

In [13]:
df_pre.head(1)

,codigo,aids,animais,dengue,hanseniase,hepatites,sadquirida,scongenita,sgestante,toxico,...,"Valor adicionado bruto da Indústria, a preços correntes (R$ 1.000)","Valor adicionado bruto dos Serviços, a preços correntes - exceto Administração, defesa, educação e saúde públicas e seguridade social (R$ 1.000)","Valor adicionado bruto da Administração, defesa, educação e saúde públicas e seguridade social, a preços correntes (R$ 1.000)","Impostos, líquidos de subsídios, sobre produtos, a preços correntes (R$ 1.000)","Produto Interno Bruto, a preços correntes (R$ 1.000)","Produto Interno Bruto per capita, a preços correntes (R$ 1,00)",populacao,latitude,longitude,regiao
0,110001,1,192,833,51,41,28,2,12,186,...,"14,441","130,220","158,948","29,351","495,629","21,600.73",22945,-11.9287,-61.9958,N


Total de nascimentos, mortalidade geral e mortalidade infantil entre 2015 e 2019 somados em uma única coluna.

In [14]:
df_pre["nascimentos"] = df_pre[["nascimentos_2015", "nascimentos_2016", "nascimentos_2017", "nascimentos_2018", "nascimentos_2019"]].sum(axis=1)
df_pre["mortalidade_geral"] = df_pre[["mortalidade_geral_2015", "mortalidade_geral_2016", "mortalidade_geral_2017", "mortalidade_geral_2018", "mortalidade_geral_2019"]].sum(axis=1)
df_pre["mortalidade_infantil"] = df_pre[["mortalidade_infantil_2015", "mortalidade_infantil_2016", "mortalidade_infantil_2017", "mortalidade_infantil_2018", "mortalidade_infantil_2019"]].sum(axis=1)
df_pre = df_pre.drop(["nascimentos_2015", "nascimentos_2016", "nascimentos_2017", "nascimentos_2018", "nascimentos_2019"], axis=1)
df_pre = df_pre.drop(["mortalidade_geral_2015", "mortalidade_geral_2016", "mortalidade_geral_2017", "mortalidade_geral_2018", "mortalidade_geral_2019"], axis=1)
df_pre = df_pre.drop(["mortalidade_infantil_2015", "mortalidade_infantil_2016", "mortalidade_infantil_2017", "mortalidade_infantil_2018", "mortalidade_infantil_2019"], axis=1)
df_pre

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

,codigo,aids,animais,dengue,hanseniase,hepatites,sadquirida,scongenita,sgestante,toxico,...,"Impostos, líquidos de subsídios, sobre produtos, a preços correntes (R$ 1.000)","Produto Interno Bruto, a preços correntes (R$ 1.000)","Produto Interno Bruto per capita, a preços correntes (R$ 1,00)",populacao,latitude,longitude,regiao,nascimentos,mortalidade_geral,mortalidade_infantil
0,110001,1,192,833,51,41,28,2,12,186,...,"29,351","495,629","21,600.73",22945,-11.92870,-61.9958,N,1828,659,23
1,110037,1,199,90,86,12,13,1,2,31,...,"11,248","288,681","21,802.07",13241,-12.14050,-61.8379,N,803,252,4
2,110040,5,62,65,43,66,18,2,11,18,...,"18,150","358,464","16,728.75",21428,-9.71470,-63.3193,N,1154,369,24
3,110034,5,78,126,24,8,16,1,7,35,...,"10,423","260,348","18,065.91",14411,-11.34670,-62.2852,N,911,414,8
4,110002,121,252,198,224,240,296,8,89,321,...,"284,398","2,578,830","23,908.38",107863,-9.90612,-63.0330,N,8580,2557,111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,522145,0,24,115,12,0,0,0,1,3,...,"2,444","51,875","14,821.32",3500,-13.50830,-48.7421,CO,155,113,2
5566,522157,0,17,137,11,1,1,1,2,0,...,"1,956","47,960","16,804.54",2854,-14.28400,-49.9205,CO,120,88,1
5567,220460,0,0,14,6,2,0,1,2,1,...,"1,292","30,136","7,773.10",3877,-5.98900,-42.5601,NE,230,115,1
5568,220590,0,0,21,5,0,7,3,3,3,...,"2,888","51,080","9,549.44",5349,-8.00434,-43.8723,NE,344,148,5


Colunas com nomes com muitos caracteres são renomeadas.

In [15]:
df_pre = df_pre.rename({"Região Metropolitana": "metropolitana", "Amazônia Legal": "amazonia", "Semiárido": "semiarido", 
                        "Valor adicionado bruto da Agropecuária, preços correntes(R$ 1.000)": "agropecuaria", "Valor adicionado bruto da Indústria, a preços correntes (R$ 1.000)": "industria",
                        "Valor adicionado bruto dos Serviços, a preços correntes  - exceto Administração, defesa, educação e saúde públicas e seguridade social (R$ 1.000)": "servicos", 
                        "Valor adicionado bruto da Administração, defesa, educação e saúde públicas e seguridade social, a preços correntes (R$ 1.000)": "servicos_basicos", 
                        "Impostos, líquidos de subsídios, sobre produtos, a preços correntes (R$ 1.000)": "impostos", "Produto Interno Bruto, a preços correntes (R$ 1.000)": "pib", 
                        "Produto Interno Bruto per capita, a preços correntes (R$ 1,00)": "per_capita"}, axis=1)

In [16]:
df_pre.head(1)

,codigo,aids,animais,dengue,hanseniase,hepatites,sadquirida,scongenita,sgestante,toxico,...,impostos,pib,per_capita,populacao,latitude,longitude,regiao,nascimentos,mortalidade_geral,mortalidade_infantil
0,110001,1,192,833,51,41,28,2,12,186,...,"29,351","495,629","21,600.73",22945,-11.9287,-61.9958,N,1828,659,23


Substituição de sim, não, nulo ou em branco por 0 (não) ou 1 (sim).

In [17]:
df_pre.loc[df_pre["metropolitana"].notna(), "metropolitana"] = 1
df_pre.loc[df_pre["metropolitana"].isna(), "metropolitana"] = 0
df_pre.loc[df_pre["amazonia"]=="Sim", "amazonia"] = 1
df_pre.loc[df_pre["amazonia"]=="Não", "amazonia"] = 0
df_pre.loc[df_pre["semiarido"]=="Sim", "semiarido"] = 1
df_pre.loc[df_pre["semiarido"]=="Não", "semiarido"] = 0


In [18]:
df_pre

,codigo,aids,animais,dengue,hanseniase,hepatites,sadquirida,scongenita,sgestante,toxico,...,impostos,pib,per_capita,populacao,latitude,longitude,regiao,nascimentos,mortalidade_geral,mortalidade_infantil
0,110001,1,192,833,51,41,28,2,12,186,...,"29,351","495,629","21,600.73",22945,-11.92870,-61.9958,N,1828,659,23
1,110037,1,199,90,86,12,13,1,2,31,...,"11,248","288,681","21,802.07",13241,-12.14050,-61.8379,N,803,252,4
2,110040,5,62,65,43,66,18,2,11,18,...,"18,150","358,464","16,728.75",21428,-9.71470,-63.3193,N,1154,369,24
3,110034,5,78,126,24,8,16,1,7,35,...,"10,423","260,348","18,065.91",14411,-11.34670,-62.2852,N,911,414,8
4,110002,121,252,198,224,240,296,8,89,321,...,"284,398","2,578,830","23,908.38",107863,-9.90612,-63.0330,N,8580,2557,111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,522145,0,24,115,12,0,0,0,1,3,...,"2,444","51,875","14,821.32",3500,-13.50830,-48.7421,CO,155,113,2
5566,522157,0,17,137,11,1,1,1,2,0,...,"1,956","47,960","16,804.54",2854,-14.28400,-49.9205,CO,120,88,1
5567,220460,0,0,14,6,2,0,1,2,1,...,"1,292","30,136","7,773.10",3877,-5.98900,-42.5601,NE,230,115,1
5568,220590,0,0,21,5,0,7,3,3,3,...,"2,888","51,080","9,549.44",5349,-8.00434,-43.8723,NE,344,148,5


In [19]:
df_pre["regiao"] = df_pre["regiao"].astype("category")

In [20]:
df_pre["regiao"]

0        N
1        N
2        N
3        N
4        N
        ..
5565    CO
5566    CO
5567    NE
5568    NE
5569    NE
Name: regiao, Length: 5570, dtype: category
Categories (5, object): ['CO', 'N', 'NE', 'S', 'SE']

Transformando as 5 regiões em valores numéricos entre 0 e 1.

In [21]:
df_pre["regiao"].cat.codes

0       1
1       1
2       1
3       1
4       1
       ..
5565    0
5566    0
5567    2
5568    2
5569    2
Length: 5570, dtype: int8

In [22]:
df_pre["estado"] = df_pre["estado"].astype("category")
df_pre["metropolitana"] = df_pre["metropolitana"].astype("category")
df_pre["amazonia"] = df_pre["amazonia"].astype("category")
df_pre["semiarido"] = df_pre["semiarido"].astype("category")

In [23]:
df_pre["estado"]

0       RO
1       RO
2       RO
3       RO
4       RO
        ..
5565    GO
5566    GO
5567    PI
5568    PI
5569    PB
Name: estado, Length: 5570, dtype: category
Categories (27, object): ['AC', 'AL', 'AM', 'AP', ..., 'SC', 'SE', 'SP', 'TO']

Transformando os 27 estados em valores numéricos entre 0 e 26.

In [24]:
df_pre["estado"].cat.codes

0       20
1       20
2       20
3       20
4       20
        ..
5565     8
5566     8
5567    16
5568    16
5569    14
Length: 5570, dtype: int8

In [25]:
df_pre["pib"]

0          495,629 
1          288,681 
2          358,464 
3          260,348 
4        2,578,830 
           ...     
5565        51,875 
5566        47,960 
5567        30,136 
5568        51,080 
5569        60,564 
Name: pib, Length: 5570, dtype: object

In [26]:
df_pre[df_pre["agropecuaria"] == " -   "]

,codigo,aids,animais,dengue,hanseniase,hepatites,sadquirida,scongenita,sgestante,toxico,...,impostos,pib,per_capita,populacao,latitude,longitude,regiao,nascimentos,mortalidade_geral,mortalidade_infantil
2720,330320,182,11,479,33,60,312,227,141,79,...,"170,928","2,904,426","17,875.04",162485,-22.8061,-43.4237,SE,9461,7691,146
2772,350060,1,122,46,1,6,2,0,0,12,...,"7,797","145,907","42,279.55",3451,-22.5982,-47.8738,SE,137,167,2


Após erros (apagados do notebook), identificado que na coluna agropecuária alguns valores estavam faltando. Os dois valores serão substituídos por 0 porque conferi também nos outros anos diretamente na planilha do IBGE e também nos outros anos (além do ano de 2019 que foi preservado entre o tcc1 e este notebook), também eram 0.

In [27]:
df_pre["pib"] = df_pre["pib"].str.replace(",", "").astype(float)
df_pre["agropecuaria"] = df_pre["agropecuaria"].str.replace(",", "").replace(" ", "").replace(" -   ", "0").astype(float)
df_pre["industria"] = df_pre["industria"].str.replace(",", "").astype(float)
df_pre["servicos"] = df_pre["servicos"].str.replace(",", "").astype(float)
df_pre["servicos_basicos"] = df_pre["servicos_basicos"].str.replace(",", "").astype(float)
df_pre["impostos"] = df_pre["impostos"].str.replace(",", "").astype(float)
df_pre["per_capita"] = df_pre["per_capita"].str.replace(",", "").astype(float)

In [28]:
df_pre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 34 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   codigo                     5570 non-null   int64   
 1   aids                       5570 non-null   int64   
 2   animais                    5570 non-null   int64   
 3   dengue                     5570 non-null   int64   
 4   hanseniase                 5570 non-null   int64   
 5   hepatites                  5570 non-null   int64   
 6   sadquirida                 5570 non-null   int64   
 7   scongenita                 5570 non-null   int64   
 8   sgestante                  5570 non-null   int64   
 9   toxico                     5570 non-null   int64   
 10  tuberculose                5570 non-null   int64   
 11  autoprovocada_sim          5570 non-null   int64   
 12  autoprovocada_não          5570 non-null   int64   
 13  violencia                  5570 n

In [29]:
df_pre = df_pre.rename({"total_de_estabelecimentos": "estabelecimentos", "total_de_profissionais":"profissionais"}, axis=1)

In [30]:
df_pre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   codigo                5570 non-null   int64   
 1   aids                  5570 non-null   int64   
 2   animais               5570 non-null   int64   
 3   dengue                5570 non-null   int64   
 4   hanseniase            5570 non-null   int64   
 5   hepatites             5570 non-null   int64   
 6   sadquirida            5570 non-null   int64   
 7   scongenita            5570 non-null   int64   
 8   sgestante             5570 non-null   int64   
 9   toxico                5570 non-null   int64   
 10  tuberculose           5570 non-null   int64   
 11  autoprovocada_sim     5570 non-null   int64   
 12  autoprovocada_não     5570 non-null   int64   
 13  violencia             5570 non-null   int64   
 14  estabelecimentos      5570 non-null   int64   
 15  prof

# SEGUNDO INTERVALO ENTRE NOTEBOOKS

In [31]:
from google.colab import files
df_pre.to_csv('df_pre.csv', encoding = 'utf-8-sig', index=False) 
files.download('df_pre.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>